In [35]:
import nibabel as nb
import numpy as np
# import scipy.io
import scipy
from scipy import stats
import math
import pandas as pd
import statsmodels.api as sm
from ipynb.fs.full.utils import *

In [36]:
regressor_mean_magnitude_motion_vectors = np.load('E:/FPP_files_and_codes/FPP_files/dense_optical_flow_vectors/mean_magnitude_vectors_in_each_second.npy')
regressor_scene_transitions = np.load('E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_absdif_intensity_pixels/regressor_mean_maxdif_pixel_intensitiy.npy')


In [37]:
file_path = 'E:/FPP_files_and_codes/FPP_files/relabeled_seconds/corrected_GPT_labeling/final_labels.csv'

RelabelDataframe = pd.read_csv(file_path)

In [38]:
RelabelDictionary = {}
for index, row in RelabelDataframe.iterrows():
    if index != 0:
        RelabelDictionary[row['prv_label']] = row['new_label']
RelabelDictionary[''] = float('nan')

In [39]:
labels_list_1 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie1_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_1.append(labels)
labels_list_2 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie2_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_2.append(labels)
labels_list_3 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie3_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_3.append(labels)
labels_list_4 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie4_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_4.append(labels)
labels_list = labels_list_1 + labels_list_2 + labels_list_3  + labels_list_4
print(len(labels_list))

3655


In [40]:
RelabeldFrames = [set() for i in range(3655)]
for i in range(3655):
    for j in labels_list[i]:
        RelabeldFrames[i].add(str(RelabelDictionary[j]))

In [41]:
RelabeldFrames

[{'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'B

In [42]:
regressor_blank = np.ones(3655)
for i,labels in enumerate(RelabeldFrames):
    if 'nan' in labels:
        regressor_blank[i: i+6] = 0

In [43]:
regressor_blank = regressor_blank.astype(bool)

In [44]:
regressor_blank

array([False, False, False, ..., False, False, False])

In [45]:
delay = 4
shifted_regressor_mean_magnitude_motion_vectors = np.roll(regressor_mean_magnitude_motion_vectors, delay)
shifted_regressor_scene_transitions = np.roll(regressor_scene_transitions, delay)
shifted_regressor_blank = np.roll(regressor_blank, delay)

In [46]:
left_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [47]:
shifted_regressor_blank.sum()

3097

In [48]:
shifted_regressor_scene_transitions[shifted_regressor_blank].shape

(3097,)

In [49]:
shifted_regressor_scene_transitions_without_blank = shifted_regressor_scene_transitions[shifted_regressor_blank]
shifted_regressor_mean_magnitude_motion_vectors_without_blank = shifted_regressor_mean_magnitude_motion_vectors[shifted_regressor_blank]
right_data_without_blank = right_data[:,shifted_regressor_blank]

In [50]:
shifted_regressor_scene_transitions_without_blank

array([ 0.25756655,  0.17257957,  0.16663821, ..., 49.26583526,
       25.7679755 , 43.16022618])

In [51]:
shifted_regressor_mean_magnitude_motion_vectors_without_blank

array([0.01746612, 0.01238639, 0.01103873, ..., 0.81531066, 1.23551083,
       1.14394557])

In [52]:
shifted_regressor_mean_magnitude_motion_vectors_2d = shifted_regressor_mean_magnitude_motion_vectors_without_blank.reshape(1, -1)
shifted_regressor_scene_transitions_2d = shifted_regressor_scene_transitions_without_blank.reshape(1, -1)

X = np.concatenate((shifted_regressor_mean_magnitude_motion_vectors_2d, shifted_regressor_scene_transitions_2d), axis=0).T

coefficients_regressor_motion_right = np.zeros(32492)
coefficients_regrssor_transition_right = np.zeros(32492)

pvalue_coefficients_regressor_motion_right = np.zeros(32492)
pvalue_coefficients_regrssor_transition_right = np.zeros(32492)


zero = np.zeros(3655)

for (i,vertex_data) in enumerate(right_data_without_blank):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2'])
    df['y'] = y

    X = sm.add_constant(df[['X1', 'X2']])
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):

        if np.any(np.isnan(y)) or np.any(np.isnan(X)):
            continue

        # Check for zero variance in y
        if np.var(y) == 0:
            continue

        result = model.fit()
        coefficients = result.params

        coefficients_regressor_motion_right[i] = coefficients['X1']
        coefficients_regrssor_transition_right[i] = coefficients['X2']
        
        p_values = result.pvalues
        pvalue_coefficients_regressor_motion_right[i] = p_values['X1']
        pvalue_coefficients_regrssor_transition_right[i] = p_values['X2']

In [32]:
(coefficients_regressor_motion_right == 'NAN').any()

False

In [63]:
making_beta_and_sig_map_right(coefficients_regrssor_transition_right, pvalue_coefficients_regrssor_transition_right,\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/beta_regressor_transition_regressout_motion_without_blank.R.func.gii',\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/significance_beta_regressor_transition_regressout_motion_without_blank.R.func.gii')

making_beta_and_sig_map_right(coefficients_regressor_motion_right, pvalue_coefficients_regressor_motion_right,\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/beta_regressor_motion_regressout_transition_without_blank.R.func.gii',\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/significance_beta_regressor_motion_regressout_transition_without_blank.R.func.gii')

e:\FPP_files_and_codes\FPP_codebase\Figure6_analysis\Final_analysis\utils.ipynb:17: RuntimeWarning: divide by zero encountered in log
  "outputs": [],
e:\FPP_files_and_codes\FPP_codebase\Figure6_analysis\Final_analysis\utils.ipynb:17: RuntimeWarning: invalid value encountered in multiply
  "outputs": [],


In [58]:
right_data_without_blank[7] == 0

array([ True,  True,  True, ...,  True,  True,  True])

In [64]:
left_data_without_blank = left_data[:,shifted_regressor_blank]

In [65]:
left_data_without_blank.shape

(32492, 3097)

In [66]:
shifted_regressor_mean_magnitude_motion_vectors_2d = shifted_regressor_mean_magnitude_motion_vectors_without_blank.reshape(1, -1)
shifted_regressor_scene_transitions_2d = shifted_regressor_scene_transitions_without_blank.reshape(1, -1)

X = np.concatenate((shifted_regressor_mean_magnitude_motion_vectors_2d, shifted_regressor_scene_transitions_2d), axis=0).T

coefficients_regressor_motion_left = np.zeros(32492)
coefficients_regrssor_transition_left = np.zeros(32492)

pvalue_coefficients_regressor_motion_left = np.zeros(32492)
pvalue_coefficients_regrssor_transition_left = np.zeros(32492)


zero = np.zeros(3655)

for (i,vertex_data) in enumerate(left_data_without_blank):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2'])
    df['y'] = y

    X = sm.add_constant(df[['X1', 'X2']])
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):

        if np.any(np.isnan(y)) or np.any(np.isnan(X)):
            continue

        # Check for zero variance in y
        if np.var(y) == 0:
            continue
        result = model.fit()
        coefficients = result.params

        coefficients_regressor_motion_left[i] = coefficients['X1']
        coefficients_regrssor_transition_left[i] = coefficients['X2']
        
        p_values = result.pvalues
        pvalue_coefficients_regressor_motion_left[i] = p_values['X1']
        pvalue_coefficients_regrssor_transition_left[i] = p_values['X2']

In [67]:
making_beta_and_sig_map_left(coefficients_regrssor_transition_left, pvalue_coefficients_regrssor_transition_left,\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/beta_regressor_transition_regressout_motion_without_blank.L.func.gii',\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/significance_beta_regressor_transition_regressout_motion_without_blank.L.func.gii')

making_beta_and_sig_map_left(coefficients_regressor_motion_left, pvalue_coefficients_regressor_motion_left,\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/beta_regressor_motion_regressout_transition_without_blank.L.func.gii',\
                              'E:/FPP_files_and_codes/FPP_files/scene_transition/regressor_transition_regressout_motion/significance_beta_regressor_motion_regressout_transition_without_blank.L.func.gii')

e:\FPP_files_and_codes\FPP_codebase\Figure6_analysis\Final_analysis\utils.ipynb:32: RuntimeWarning: divide by zero encountered in log
  "    temp_r.darrays[0].data = float32_significance_beta\n",
e:\FPP_files_and_codes\FPP_codebase\Figure6_analysis\Final_analysis\utils.ipynb:32: RuntimeWarning: invalid value encountered in multiply
  "    temp_r.darrays[0].data = float32_significance_beta\n",


In [20]:
print(model.scaletype)

None


In [25]:
model.scale

0.057004753896474525

In [28]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 3655
Model:                            GLM   Df Residuals:                     3652
Model Family:                Gaussian   Df Model:                            2
Link Function:               Identity   Scale:                        0.057005
Method:                          IRLS   Log-Likelihood:                 50.374
Date:                Fri, 12 Jul 2024   Deviance:                       208.18
Time:                        22:35:45   Pearson chi2:                     208.
No. Iterations:                     3   Pseudo R-squ. (CS):           0.003890
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       -1.61e-10      0.004  -4.08e-08      1.000      -0.008       0.008
X1            -0.0148      0.004     -3.647      0.000      -0.023      -0.007
X2            -0.0003      0.004     -0.070      0.944      -0.008       0.008
==============================================================================
"""